<a href="https://colab.research.google.com/github/NathalyDM/genetifics/blob/main/ImageProcessingTutorials/ImageMultipleClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clasificacion de imagenes en tres clases**

## **1. Importar las bibliotecas necesarias**
Importa las bibliotecas y módulos necesarios para ejecutar el código.
os y numpy son bibliotecas estándar de Python para operaciones del sistema y operaciones matemáticas, respectivamente.
tensorflow es la biblioteca principal para construir y entrenar modelos de aprendizaje profundo.
ImageDataGenerator es una clase de Keras para la aumentación de datos.
ResNet50, InceptionV3, Xception, VGG16, y VGG19 son arquitecturas de modelos preentrenados disponibles en Keras.
albumentations es una biblioteca de aumentación de imágenes rápida y flexible.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, InceptionV3, Xception, VGG16, VGG19
from tensorflow.keras import layers, models
import albumentations as A
import matplotlib.pyplot as plt

In [ ]:
def plot_history(history, model_name):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_images(images, augmented_images):
    plt.figure(figsize=(10, 4))
    for i in range(4):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        plt.title('Original')
        plt.axis('off')

        plt.subplot(2, 4, i + 5)
        plt.imshow(augmented_images[i])
        plt.title('Augmented')
        plt.axis('off')
    plt.tight_layout()
    plt.show()


## **2. Configurar los directorios**
- Define las rutas a los directorios de entrenamiento y validación.
- `train_dir` y `val_dir` deben contener subcarpetas para cada clase.



In [ ]:
base_dir = 'path_to_your_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

"""
dataset
│
└───train
│   └───control
│   │   └───image1.jpg
│   │   └───image2.jpg
│   │   ...
│   └───covid
│   │   └───image1.jpg
│   │   └───image2.jpg
│   │   ...
│   └───pneumonia-viral
│       └───image1.jpg
│       └───image2.jpg
│       ...
│
└───val
    └───control
    │   └───image1.jpg
    │   └───image2.jpg
    │   ...
    └───covid
    │   └───image1.jpg
    │   └───image2.jpg
    │   ...
    └───pneumonia-viral
        └───image1.jpg
        └───image2.jpg
        ...
"""

## **3. Preprocesamiento y Aumentación de Datos con Albumentations**
Define las rutas a los directorios de entrenamiento y validación.
`train_dir` y `val_dir` deben contener subcarpetas para cada clase.

- Define una función para transformar las imágenes usando la biblioteca Albumentations.
- Realiza volteo horizontal, ajuste de brillo y contraste, y normalización de las imágenes.

In [ ]:
def albumentations_transform(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    image = transform(image=image)['image']
    return image

### **3.1. Obtener y visualizar 4 imágenes aleatorias y sus versiones aumentadas**

In [ ]:
# Obtener un lote de imágenes del generador de entrenamiento
images, _ = next(train_generator)

# Aplicar la función de transformación de Albumentations a cada imagen
augmented_images = [albumentations_transform(image) for image in images]

# Visualizar las imágenes originales y aumentadas
plot_images(images, augmented_images)

## **4. Generador de Datos**

- Define una función para crear un generador de datos que leerá las imágenes de un directorio y aplicará las transformaciones definidas anteriormente.
- El generador de datos es útil para leer y procesar imágenes en lotes durante el entrenamiento y la evaluación del modelo.

In [ ]:
def data_generator(directory, batch_size):
    datagen = ImageDataGenerator(preprocessing_function=albumentations_transform)
    generator = datagen.flow_from_directory(
        directory,
        class_mode='categorical',
        batch_size=batch_size
    )
    return generator

## **5. Crear Generadores de Datos**

Crea generadores de datos para los conjuntos de entrenamiento y validación usando la función definida anteriormente.

In [ ]:
train_generator = data_generator(train_dir, batch_size=32)
val_generator = data_generator(val_dir, batch_size=32)

## **5. Construir, Compilar, Entrenar y Evaluar los Modelos**

### **5.1 ResNet50**

In [ ]:
base_model = ResNet50(input_shape=(None, None, 3), include_top=False, weights='imagenet')
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)
loss, accuracy = model.evaluate(val_generator)
plot_history(history, 'ResNet50')
model.save('ResNet50_model.h5')
print(f'ResNet50 - Loss: {loss}, Accuracy: {accuracy}')

### **5.2 InceptionV3**

In [ ]:
base_model = InceptionV3(input_shape=(None, None, 3), include_top=False, weights='imagenet')
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)
loss, accuracy = model.evaluate(val_generator)
plot_history(history, 'InceptionV3')
model.save('InceptionV3_model.h5')
print(f'InceptionV3 - Loss: {loss}, Accuracy: {accuracy}')

### **5.3 Xception**

In [ ]:
base_model = Xception(input_shape=(None, None, 3), include_top=False, weights='imagenet')
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)
loss, accuracy = model.evaluate(val_generator)
plot_history(history, 'Xception')
model.save('Xception_model.h5')
print(f'Xception - Loss: {loss}, Accuracy: {accuracy}')

### **5.4 VGG16**

In [ ]:
base_model = VGG16(input_shape=(None, None, 3), include_top=False, weights='imagenet')
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)
loss, accuracy = model.evaluate(val_generator)
plot_history(history, 'VGG16')
model.save('VGG16_model.h5')
print(f'VGG16 - Loss: {loss}, Accuracy: {accuracy}')

### **5.5 VGG19**

In [ ]:
base_model = VGG19(input_shape=(None, None, 3), include_top=False, weights='imagenet')
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)
loss, accuracy = model.evaluate(val_generator)
plot_history(history, 'VGG19')
model.save('VGG19_model.h5')
print(f'VGG19 - Loss: {loss}, Accuracy: {accuracy}')

## **6. Sección de Inferencia**

- `load_random_images`: Esta función carga rutas de imágenes aleatorias del directorio de datos proporcionado.
-`preprocess_and_load_image`: Esta función carga, preprocesa y transforma una imagen al formato correcto para la inferencia del modelo.
-Luego, para cada modelo entrenado, cargamos el modelo, preprocesamos las imágenes aleatorias cargadas y realizamos inferencias.
-Imprimimos las clases predichas junto con la confianza para cada imagen usando cada modelo.

In [ ]:
def load_random_images(directory, num_images):
    all_images = []
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if os.path.isdir(subdir_path):
            all_images.extend([os.path.join(subdir_path, file) for file in os.listdir(subdir_path)])
    selected_images = random.sample(all_images, num_images)
    return selected_images

def preprocess_and_load_image(img_path):
    img = k_image.load_img(img_path, target_size=(224, 224))  # adjust target_size according to your model input size
    img_array = k_image.img_to_array(img)
    img_array = albumentations_transform(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to represent a batch of size 1
    return img_array

# Load a few random images
num_images = 5
random_images = load_random_images(train_dir, num_images)

# Load the trained models and perform inference
model_names = ['ResNet50', 'InceptionV3', 'Xception', 'VGG16', 'VGG19']
for model_name in model_names:
    print(f'Performing inference using {model_name} model...')
    model = load_model(f'{model_name}_model.h5')  # Load the trained model
    for img_path in random_images:
        img_array = preprocess_and_load_image(img_path)
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)
        print(f'Image {img_path} is predicted as class {predicted_class[0]} with confidence {predictions[0][predicted_class[0]]:.2f}')
    print('\n' + '='*80 + '\n')  # Print separator between different models' predictions